[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/sdxl-line-art-style-transfer-jupyter/blob/main/sdxl_line_art_style_transfer_comfyui_jupyter)

In [ ]:
!pip install opencv-contrib-python imageio imageio-ffmpeg ffmpeg-python av runpod
!pip install torchsde einops diffusers transformers accelerate peft timm kornia scikit-image matplotlib blend-modes segment-anything spandrel wget pydantic aiohttp
!git clone https://github.com/comfyanonymous/ComfyUI /content/ComfyUI
!git clone https://github.com/ltdrdata/ComfyUI-Manager /content/ComfyUI/custom_nodes/ComfyUI-Manager
!git clone https://github.com/sipherxyz/comfyui-art-venture /content/ComfyUI/custom_nodes/comfyui-art-venture
!git clone https://github.com/chflame163/ComfyUI_LayerStyle /content/ComfyUI/custom_nodes/ComfyUI_LayerStyle
!git clone https://github.com/cubiq/ComfyUI_essentials /content/ComfyUI/custom_nodes/ComfyUI_essentials
!git clone https://github.com/Kosinkadink/ComfyUI-Advanced-ControlNet /content/ComfyUI/custom_nodes/ComfyUI-Advanced-ControlNet
!git clone https://github.com/cubiq/ComfyUI_IPAdapter_plus /content/ComfyUI/custom_nodes/ComfyUI_IPAdapter_plus
!git clone https://github.com/Fannovel16/comfyui_controlnet_aux /content/ComfyUI/custom_nodes/comfyui_controlnet_aux
!apt install aria2 -qqy
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/misri/leosamsHelloworldXL_helloworldXL70/resolve/main/leosamsHelloworldXL_helloworldXL70.safetensors -d /content/ComfyUI/models/checkpoints/sdxl -o leosamsHelloworldXL_helloworldXL70.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/alvdansen/midsommarcartoon/resolve/main/araminta_k_midsommar_cartoon.safetensors -d /content/ComfyUI/models/loras/sdxl -o araminta_k_midsommar_cartoon.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/xinsir/controlnet-union-sdxl-1.0/resolve/main/diffusion_pytorch_model_promax.safetensors -d /content/ComfyUI/models/controlnet/sdxl -o controlnet-union-sdxl-1.0-promax.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/yiwangsimple/BiRefNet-general-epoch_244/resolve/main/BiRefNet-general-epoch_244.pth -d /content/ComfyUI/models/BiRefNet/pth -o BiRefNet-general-epoch_244.pth
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/h94/IP-Adapter/resolve/main/sdxl_models/ip-adapter_sdxl_vit-h.safetensors -d /content/ComfyUI/models/ipadapter -o ip-adapter_sdxl_vit-h.safetensors
!aria2c --console-log-level=error -c -x 16 -s 16 -k 1M https://huggingface.co/sharing1179/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors/resolve/main/CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors -d /content/ComfyUI/models/clip_vision -o CLIP-ViT-H-14-laion2B-s32B-b79K.safetensors

In [ ]:
!wget https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O /content/cloudflared-linux-amd64 && chmod 777 /content/cloudflared-linux-amd64
import atexit, requests, subprocess, time, re, os
from random import randint
from threading import Timer
from queue import Queue
def cloudflared(port, metrics_port, output_queue):
    atexit.register(lambda p: p.terminate(), subprocess.Popen(['/content/cloudflared-linux-amd64', 'tunnel', '--url', f'http://127.0.0.1:{port}', '--metrics', f'127.0.0.1:{metrics_port}'], stdout=subprocess.DEVNULL, stderr=subprocess.STDOUT))
    attempts, tunnel_url = 0, None
    while attempts < 10 and not tunnel_url:
        attempts += 1
        time.sleep(3)
        try:
            tunnel_url = re.search("(?P<url>https?:\/\/[^\s]+.trycloudflare.com)", requests.get(f'http://127.0.0.1:{metrics_port}/metrics').text).group("url")
        except:
            pass
    if not tunnel_url:
        raise Exception("Can't connect to Cloudflare Edge")
    output_queue.put(tunnel_url)
output_queue, metrics_port = Queue(), randint(8100, 9000)
thread = Timer(2, cloudflared, args=(8188, metrics_port, output_queue))
thread.start()
thread.join()
tunnel_url = output_queue.get()
os.environ['webui_url'] = tunnel_url
print(tunnel_url)
%cd /content/ComfyUI
!python main.py